# visualize balloon dataset

- 按照视频中 notebook 步骤，可视化数据集和标签


In [1]:
import os
import shutil

from tqdm import tqdm

PROJECT_ROOT = "/home/louzekun/projects/mmlab/mmdet"
os.chdir(PROJECT_ROOT)
os.getcwd()

'/mnt/data/ai4phys/louzekun/projects/mmlab/mmdet'

In [2]:
!ls; ls data; ls data/balloon


balloon_RTMDet.py  configs		     mmdet_RTMDet.ipynb       outputs
basic.ipynb	   data			     mmdet_test.ipynb	      README.md
checkpoint	   mmdet_proc_dataset.ipynb  mmdet_viz_dataset.ipynb  tools
balloon  Drink_coco
train  val


## 可视化数据集

In [18]:
import matplotlib.pyplot as plt
from PIL import Image

img_dpath = "data/balloon/train/"
img_fnames = os.listdir(img_dpath)[:8]
img_fpaths = [os.path.join(img_dpath, fname) for fname in img_fnames]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, img_fpath in enumerate(img_fpaths):
    name = img_fpath.split('/')[-1].rsplit('.', 1)[0]
    img = Image.open(img_fpath).convert('RGB')
    ax = axes[idx]
    ax.imshow(img)
    ax.set_title(name)
    ax.axis('off')

fig.tight_layout()
fig = plt.gcf()
fig.savefig("outputs/train_viz.jpg")
plt.close()

![train_viz](./outputs/train_viz.jpg)

## 可视化标签

In [25]:
import json
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

train_dpath = "./data/balloon/train/"

with open(os.path.join(train_dpath, "via_region_data.json")) as f:
    anno = json.load(f)

img_ids = list(anno.keys())
img_ids_select = img_ids[:8]

fig, axes = plt.subplots(2, 4, figsize=(16, 8), dpi=100)
axes = axes.flatten()

for idx, img_id in enumerate(img_ids_select):
    img_fname = anno[img_id]['filename']
    size = anno[img_id]['size']
    img = Image.open(os.path.join(train_dpath, img_fname)).convert('RGB')

    regions = anno[img_id]['regions']
    patches = []
    for region_id, region_info in regions.items():
        x = region_info['shape_attributes']['all_points_x']
        y = region_info['shape_attributes']['all_points_y']
        points = list(zip(x,y))
        polygon = Polygon(points, closed=True)
        patches.append(polygon)
    p = PatchCollection(patches, linewidths=2, alpha=0.5, facecolor='grey', edgecolor="yellow")

    ax = axes[idx]
    ax.imshow(img)
    ax.add_collection(p)
    ax.set_title(img_fname.rsplit('.', 1)[0])
    ax.axis('off')

fig.tight_layout()
fig = plt.gcf()
fig.savefig("outputs/train_viz_polygon.jpg")
plt.close()


![train_viz_polygon](./outputs/train_viz_polygon.jpg)


# 可视化标签


In [3]:
from pycocotools.coco import COCO
import numpy
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

In [4]:
def apply_exif_orientation(image):
    _EXIF_ORIENT = 274
    if not hasattr(image, 'getexif'):
        return image

    try:
        exif = image.getexif()
    except Exception:
        exif = None

    if exif is None:
        return image

    orientation = exif.get(_EXIF_ORIENT)

    method = {
        2: Image.FLIP_LEFT_RIGHT,
        3: Image.ROTATE_180,
        4: Image.FLIP_TOP_BOTTOM,
        5: Image.TRANSPOSE,
        6: Image.ROTATE_270,
        7: Image.TRANSVERSE,
        8: Image.ROTATE_90,
    }.get(orientation)
    if method is not None:
        return image.transpose(method)
    return image

def show_bbox_only(ax, coco, anns, show_label_bbox=True, is_filling=True):
    """Show bounding box of annotations Only."""
    if len(anns) == 0:
        return

    # ax = plt.gca()
    ax.set_autoscale_on(False)

    image2color = dict()
    for cat in coco.getCatIds():
        image2color[cat] = (numpy.random.random((1, 3)) * 0.7 + 0.3).tolist()[0]

    polygons = []
    colors = []

    for ann in anns:
        color = image2color[ann['category_id']]
        bbox_x, bbox_y, bbox_w, bbox_h = ann['bbox']
        poly = [[bbox_x, bbox_y], [bbox_x, bbox_y + bbox_h],
                [bbox_x + bbox_w, bbox_y + bbox_h], [bbox_x + bbox_w, bbox_y]]
        polygons.append(Polygon(numpy.array(poly).reshape((4, 2))))
        colors.append(color)

        if show_label_bbox:
            label_bbox = dict(facecolor=color)
        else:
            label_bbox = None

        ax.text(
            bbox_x,
            bbox_y,
            '%s' % (coco.loadCats(ann['category_id'])[0]['name']),
            color='white',
            bbox=label_bbox)

    if is_filling:
        p = PatchCollection(
            polygons, facecolor=colors, linewidths=0, alpha=0.4)
        ax.add_collection(p)
    p = PatchCollection(
        polygons, facecolor='none', edgecolors=colors, linewidths=2)
    ax.add_collection(p)

In [5]:
train_dpath = "./data/balloon/train/"

coco = COCO(os.path.join(train_dpath, "balloon_train.json"))
img_ids = coco.getImgIds()
img_ids_select = img_ids[:8]

fig, axes = plt.subplots(2, 4, figsize=(16, 8), dpi=100)
axes = axes.flatten()

for idx, img_id in enumerate(img_ids_select):
    img_data = coco.loadImgs(img_id)[0]
    img_fname = img_data['file_name']
    anno_ids = coco.getAnnIds(imgIds=img_id, catIds=[], iscrowd=0)
    anno = coco.loadAnns(anno_ids)

    img = Image.open(os.path.join(train_dpath, img_fname)).convert('RGB')
    img = apply_exif_orientation(img)
    ax = axes[idx]
    ax.imshow(img)
    show_bbox_only(ax, coco, anno)
    ax.set_title(img_fname.rsplit('.', 1)[0])
    ax = axes[idx]
    ax.axis('off')

fig.tight_layout()
fig = plt.gcf()
plt.close()
fig.savefig("outputs/train_viz_coco.jpg")


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


![train_viz_coco](./outputs/train_viz_coco.jpg)
